In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
filenames = os.listdir("../text")

In [ ]:
episode = pd.read_csv("../summary_processed.csv")
episode = episode.set_index("Id")

In [8]:
from jaconv import normalize

episode = pd.read_csv("../summary_processed.csv")
episode = episode.set_index("Id")
episode["Title"] = episode["Title"].apply(normalize)

# for column in episode.columns:
#     try:
#         episode[column] = episode[column].str.strip()
#     except AttributeError:
#         continue

# episode.to_csv("../summary_processed.csv")

In [46]:
episode.loc[87]

title                                ゆる言語学ラジオ忘年会ライブ【流行語大賞決定&ひたすらエモい話】
thumbnail_url       https://i.ytimg.com/vi/poT4BzX7e_Q/sddefault.j...
video_url                 https://www.youtube.com/watch?v=poT4BzX7e_Q
publication_date                                           2021-12-28
Name: 87, dtype: object

'/Users/matsukokuumahikari/ghq/github.com/wakamenori/yuru-corpus-backend/data'

In [20]:
from tqdm import tqdm

filenames = os.listdir("../text/processing/")
# def get_text_filepath(filename: str):
#     cwd = os.getcwd()
#     parent = "/".join(os.getcwd().split("/")[:-1])
#     return os.path.join(parent, filename)

def convert_1d_to_2d(l, cols):
    return [l[i : i + cols] for i in range(0, len(l), cols)]


def read_file(filename: str):
    with open("../text/processing/" + filename, "r") as f:
        lines = f.readlines()[2:-1]
    lines = [line.replace("\n", "") for line in lines if line != "\n"]
    lines_2d = convert_1d_to_2d(lines, 2)
    return lines_2d


def extract_info(line):
    timestamp = line[0].split(".")[0]
    if len(timestamp) == 5:
        timestamp = "00:" + timestamp
    text = line[1]
    return timestamp, text


def get_episode_id(title: str):
    matched = episode[episode["Title"] == normalize(title)]
    if len(matched) == 0:
        raise ValueError(title)
    return matched.index[0]


morphemes = []

for filename in tqdm(filenames):
    if not filename.endswith("vtt"):
        continue
    episode_id = get_episode_id(".".join(filename.split(".")[:-1]))
    # print(filename, episode_id)
    lines = read_file(filename)
    for line in lines:
        timestamp, text = extract_info(line)
        record = {
            "Type": "Morpheme",
            "Id": f"{episode_id}#{timestamp}#0",
            "Token": text,
            "Speaker": ""
        }
        morphemes.append(record)

morphemes_df = pd.DataFrame(morphemes)
morphemes_df = morphemes_df.drop_duplicates(subset=["Id"])
morphemes_df.to_csv("../processing/morphemes_processing.csv", index=False)
morphemes_df

100%|██████████████████████████████████████████| 19/19 [00:00<00:00, 271.01it/s]


,Type,Id,Token,Speaker
0,Morpheme,169#00:00:00#0,【ゆる言語アクラジオ】,
1,Morpheme,169#00:00:30#0,【ゆる言語アクラジオ】,
2,Morpheme,169#00:01:00#0,【ゆる言語アクラジオ】,
3,Morpheme,169#00:01:30#0,【はじめしゃちょー】,
4,Morpheme,169#00:02:00#0,が もっと普遍的な法則があって 人はそれを手掛かりにして言語,
...,...,...,...,...
8988,Morpheme,175#00:47:54#0,励みになりますので,
8989,Morpheme,175#00:47:55#0,どうぞ引き続きよろしくお願いします,
8990,Morpheme,175#00:47:58#0,以上今回も終わりにしましょう,
8991,Morpheme,175#00:47:59#0,ありがとうございました,


In [22]:
from tqdm import tqdm
spaces = pd.read_csv("../space.csv")
parentheses = pd.read_csv("../【】.csv")

# morphemes_df = pd.read_csv("../morphemes.csv")

for space in tqdm(spaces.itertuples()):
    def retrieve(token: str):
        if token.startswith(space.text):
            return token[len(space)+1:]
        return token
    
    morphemes_df["Token"] = np.vectorize(retrieve)(morphemes_df["Token"])


for space in tqdm(parentheses.itertuples()):
    def retrieve(token: str):
        return token.replace(space.text, "")

    morphemes_df["Token"] = np.vectorize(retrieve)(morphemes_df["Token"])


def retrieve(token: str):
        return token.replace("】", "").replace("【", "")


morphemes_df["Token"] = np.vectorize(retrieve)(morphemes_df["Token"])
morphemes_df = morphemes_df[morphemes_df["Token"] != ""]
morphemes_df["Speaker"] = ""
morphemes_df = morphemes_df.sort_values("Id")
morphemes_df

11it [00:00, 42.67it/s]
46it [00:00, 61.91it/s]


,Type,Id,Token,Speaker
5186,Morpheme,167#00:00:00#0,しゃちょーエンディング,
5187,Morpheme,167#00:00:30#0,しゃちょーエンディング,
5188,Morpheme,167#00:01:01#0,しゃちょーエンディング,
5189,Morpheme,167#00:01:06#0,しゃちょーエンディング,
5190,Morpheme,167#00:01:11#0,しゃちょーエンディング,
...,...,...,...,...
8988,Morpheme,175#00:47:54#0,励みになりますので,
8989,Morpheme,175#00:47:55#0,どうぞ引き続きよろしくお願いします,
8990,Morpheme,175#00:47:58#0,以上今回も終わりにしましょう,
8991,Morpheme,175#00:47:59#0,ありがとうございました,


In [24]:
morphemes_df["Speaker"] = ""
morphemes_df = morphemes_df.sort_values("Id")
morphemes_df = morphemes_df.drop_duplicates(subset=["Id"])
morphemes_df.to_csv("../processing/morphemes_processed.csv", index=False)

In [20]:
morphemes_df[morphemes_df["Id"].str.startswith("1#")].to_csv("../test/morphemes_processed.csv", index=False)

In [13]:
morphemes_df = morphemes_df[morphemes_df["Token"] == ""]

In [12]:
err

85712    久しぶりですね
Name: Token, dtype: object

In [19]:
all = []
for row in morphemes_df["Token"].str.findall("【.*?】"):
    all.extend([morpheme for morpheme in row])
all = pd.DataFrame(all)
count = all.value_counts()
count = count[count != 1]
count.to_csv("../【】_result.csv")
count

Series([], dtype: int64)

In [20]:
space = morphemes_df["Token"].str.split(" ", expand=True)[0].value_counts()
space = space[space > 99]
space.to_csv("../space_result.csv")
space

               6412
そうですね           829
しゃちょー           752
はい              660
しゃちょーエスペラント     573
しゃちょーエンディング     526
語学ラジオ           400
うん              348
あのー             337
なるほど            329
質疑応答            278
はいはい            272
そうだね            271
正解              229
へー              206
そうそうそう          197
この文章の中で         178
はいはいはい          154
なるほどなるほど        142
お               133
ありがとうございました     129
そうなんですよ         126
なるほどね           122
そうなんだ           109
ありがとうございます      108
Name: 0, dtype: int64

In [159]:
morphemes_df[morphemes_df["Token"].str.startswith("学習")].to_csv("学習.csv")

In [74]:
import os

import pandas as pd
from dotenv import load_dotenv

load_dotenv()

import boto3

db = boto3.resource(
    "dynamodb",
    # region_name="ap-northeast-1",
    endpoint_url="http://dynamodb-local:8000",
)

table = db.Table("YuruCorpusTest")


df = pd.read_csv("data/morphemes.csv")

from dataclasses import asdict, dataclass


@dataclass
class Item:
    Type: str
    Id: str
    Token: str
    Speaker: str


for row in df.itertuples():
    item = Item(Type="morpheme", Id=str(row.Id), Token=row.Token, Speaker="")
    table.put_item(Item=asdict(item))

In [14]:
df = pd.read_csv("../morphemes_processed.csv")
dev_df = df[df["Id"].str.startswith("1#")]
dev_df.to_csv("../dev/morphemes_processed.csv")

In [2]:
df = pd.read_csv("../morphemes_processed.csv")

In [4]:
df["Id"].str.split("#", expand=True)[0].unique()

array(['1', '10', '100', '101', '102', '103', '104', '105', '106', '107',
       '108', '109', '11', '110', '111', '112', '113', '114', '115',
       '116', '117', '118', '119', '12', '120', '121', '122', '123',
       '124', '125', '126', '127', '128', '129', '13', '130', '131',
       '132', '133', '134', '135', '136', '137', '138', '139', '14',
       '140', '141', '142', '143', '144', '145', '146', '147', '148',
       '15', '153', '154', '155', '156', '157', '158', '159', '16', '160',
       '161', '162', '163', '164', '166', '17', '18', '19', '2', '20',
       '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30',
       '31', '32', '33', '35', '36', '37', '38', '39', '4', '40', '41',
       '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51',
       '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61',
       '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71',
       '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81',
  

In [3]:
a = pd.read_csv("../morphemes_processed.csv")
b = pd.read_csv("../processing/morphemes_processed.csv")

df = pd.concat([a, b])
df

,Type,Id,Token,Speaker
0,Morpheme,1#00:00:00#0,ゆる言語学ラジオ,NaN
1,Morpheme,1#00:00:03#0,はい、ということで始まってまいりました,NaN
2,Morpheme,1#00:00:05#0,ゆる言語学ラジオ,NaN
3,Morpheme,1#00:00:06#0,このラジオはですね,NaN
4,Morpheme,1#00:00:07#0,大学で言語学をガチでやっていた,NaN
...,...,...,...,...
8977,Morpheme,175#00:47:54#0,励みになりますので,NaN
8978,Morpheme,175#00:47:55#0,どうぞ引き続きよろしくお願いします,NaN
8979,Morpheme,175#00:47:58#0,以上今回も終わりにしましょう,NaN
8980,Morpheme,175#00:47:59#0,ありがとうございました,NaN


In [4]:
all_ids = df["Id"].str.split("#", expand=True)[0].unique()

In [17]:
all_ids = [int(id) for id in all_ids]

In [18]:
def analyzed(id):
    if id in list(all_ids):
        return True
    else:
        return False
    
episode["IsAnalyzed"] = np.vectorize(analyzed)(episode.index)
episode

,Title,ThumbnailUrl,VideoUrl,PublicationDate,IsAnalyzed,Channel
Id,,,,,,
1,「イルカも喋る」は大ウソ【言語学って何?】#1,https://i.ytimg.com/vi/2YY9DT4uDh0/sddefault.jpg,https://www.youtube.com/watch?v=2YY9DT4uDh0,2021-03-11,True,ゆる言語学ラジオ
2,2km先では言語が違う国があるらしい...【言語がたくさんある理由】#2,https://i.ytimg.com/vi/-Zo_0_DZrvk/sddefault.jpg,https://www.youtube.com/watch?v=-Zo_0_DZrvk,2021-03-13,True,ゆる言語学ラジオ
3,藤原不比等は「ぷぢぃぱらのぷぴちょ」だった #3,https://i.ytimg.com/vi/KItCvPD86pw/sddefault.jpg,https://www.youtube.com/watch?v=KItCvPD86pw,2021-03-15,True,ゆる言語学ラジオ
4,悶・聞・関、部首が「門」なのはどれ? #4,https://i.ytimg.com/vi/v2vY-H1FAHM/sddefault.jpg,https://www.youtube.com/watch?v=v2vY-H1FAHM,2021-03-16,True,ゆる言語学ラジオ
5,英語は荒野行動!?日本語に「時制の一致」が要らない理由 #5,https://i.ytimg.com/vi/UEc3nobDjMk/sddefault.jpg,https://www.youtube.com/watch?v=UEc3nobDjMk,2021-03-17,True,ゆる言語学ラジオ
...,...,...,...,...,...,...
171,チョムスキーが提唱した「ふたまたニョキニョキ理論」【生成文法3】#169,https://i.ytimg.com/vi/CYxGKxBZApE/sddefault.jpg,https://www.youtube.com/watch?v=CYxGKxBZApE,2022-10-18,True,ゆる言語学ラジオ
172,言語とはブロッコリーである【生成文法4】#170,https://i.ytimg.com/vi/5Y-nTXVT9hk/sddefault.jpg,https://www.youtube.com/watch?v=5Y-nTXVT9hk,2022-10-21,True,ゆる言語学ラジオ
173,学んだ理由はロックバンド。言語学者のパンクな人生。【嶋村先生雑談回】#171,https://i.ytimg.com/vi/OK-a6R0wa0o/sddefault.jpg,https://www.youtube.com/watch?v=OK-a6R0wa0o,2022-10-25,True,ゆる言語学ラジオ


In [22]:
episode["IsAnalyzed"].sum()


169

In [23]:
episode.to_csv("../processing/summary_processed.csv")